# PVDF + 水の溶媒和系の作成デモ（Packmol版）

このノートブックでは、PVDF（ポリフッ化ビニリデン）構造に
水分子を充填して溶媒和系を作成する方法を示します。

**★ Packmolを使用した高速・高品質な充填**

## 主な機能
1. PVDF構造ファイルの読み込み
2. 密度指定による水分子数の自動計算
3. Packmolによる最適配置（重なりなし）
4. 構造セルサイズと水充填領域を個別指定可能
5. 初期構造の保存
6. （オプション）MDシミュレーションでの緩和

## 応用例
- PVDF膜の含水状態のシミュレーション
- PVDF/水界面の反応性研究
- 電解質溶液とPVDFの相互作用

## Packmolの利点
- ASEのランダム配置より高速
- 原子の重なりがない高品質な初期構造
- PVDFのような密な構造でも安定

In [ ]:
import numpy as np
import sys
from pathlib import Path
from ase.io import read, write
from ase import units

# プロジェクトのutilsをインポート（Packmol版）
sys.path.append(str(Path.cwd().parent / "LiB2_structure_ipynb"))
from utils.packmol_utils import (
    fill_box_with_packmol,
    estimate_required_molecules,
    check_packmol_command
)

# Packmolの確認
if check_packmol_command():
    print("✓ Packmolコマンドが利用可能です")
else:
    print("✗ 警告: Packmolコマンドが見つかりません")
    print("  Packmolをインストールしてください: https://m3g.github.io/packmol/")

print("✓ インポート完了")

## 1. 設定パラメータ

In [ ]:
CONFIG = {
    # 入力ファイル（reaxFF_H2O_PVDF_Al_shrink.ipynbと同じディレクトリ）
    'pvdf_structure_path': '/home/jovyan/Kaori/MD/LiB_2/structure/PVDF_only_shrunk.cif',
    
    # 水の密度設定
    'water_density_g_cm3': 0.85,  # 0.85 g/cm³ (PVDFの空隙が少ない可能性を考慮)
    
    # Packmolパラメータ
    'tolerance': 2.2,  # 原子間の最小距離 (Å) - PVDFは密なので少し大きめ
    'packmol_seed': 54321,  # 乱数シード（再現性のため）
    
    # セルサイズ設定（オプション）
    # None の場合は構造ファイルのセルサイズを使用
    'structure_cell_size': None,  # 構造全体のセルサイズ [a, b, c] または float（立方体）
    'water_fill_cell_size': None,  # 水を充填する領域のサイズ [a, b, c] または float（立方体）
    
    # 使用例:
    # 'structure_cell_size': [30.0, 30.0, 30.0],  # 30x30x30 Åのセル
    # 'water_fill_cell_size': [20.0, 20.0, 20.0],  # 20x20x20 Åの領域に水を充填
    # または
    # 'structure_cell_size': 30.0,  # 30x30x30 Åの立方体セル
    # 'water_fill_cell_size': 20.0,  # 20x20x20 Åの立方体領域に水を充填
    
    # 出力ディレクトリ
    'output_dir': 'solvation_results',
}

# 出力ディレクトリの作成
output_dir = Path(CONFIG['output_dir'])
output_dir.mkdir(exist_ok=True)

print("設定:")
for key, value in CONFIG.items():
    print(f"  {key}: {value}")

## 2. PVDF構造の読み込み

In [ ]:
pvdf_path = CONFIG['pvdf_structure_path']

try:
    pvdf_atoms = read(pvdf_path)
    print(f"✓ PVDF構造を読み込みました: {pvdf_path}")
    print(f"\n構造情報:")
    print(f"  組成: {pvdf_atoms.get_chemical_formula()}")
    print(f"  原子数: {len(pvdf_atoms)}")
    print(f"  セルパラメータ: {pvdf_atoms.cell.cellpar()}")
    print(f"  体積: {pvdf_atoms.get_volume():.2f} Å³")
    
    # C, H, F の数を数える
    symbols = pvdf_atoms.get_chemical_symbols()
    n_c = symbols.count('C')
    n_h = symbols.count('H')
    n_f = symbols.count('F')
    print(f"\n原子数内訳:")
    print(f"  C: {n_c}")
    print(f"  H: {n_h}")
    print(f"  F: {n_f}")
    print(f"  → PVDFモノマー数 (CH₂CF₂): 約 {min(n_c//2, n_h, n_f//2)}")
    
except FileNotFoundError:
    print(f"✗ エラー: ファイルが見つかりません: {pvdf_path}")
    print(f"\nヒント: 以下のいずれかのファイルを使用してください:")
    print(f"  - PVDF_only_shrunk.cif")
    print(f"  - その他のPVDF構造ファイル")
    raise

## 3. 必要な水分子数の見積もり

In [ ]:
# 水を充填する領域の体積を計算
if CONFIG['water_fill_cell_size'] is not None:
    # 明示的に指定された場合
    if isinstance(CONFIG['water_fill_cell_size'], (int, float)):
        water_box_size = CONFIG['water_fill_cell_size'] ** 3
    else:
        water_box_size = np.prod(CONFIG['water_fill_cell_size'])
    print(f"水充填領域: {CONFIG['water_fill_cell_size']} (指定値)")
else:
    # 構造ファイルのセルサイズを使用
    if CONFIG['structure_cell_size'] is not None:
        if isinstance(CONFIG['structure_cell_size'], (int, float)):
            water_box_size = CONFIG['structure_cell_size'] ** 3
        else:
            water_box_size = np.prod(CONFIG['structure_cell_size'])
    else:
        water_box_size = pvdf_atoms.get_volume()
    print(f"水充填領域: 構造ファイルのセルサイズと同じ")

# 水分子のモル質量
water_molar_mass = 18.015  # g/mol

# 必要な水分子数の見積もり
n_molecules_required = estimate_required_molecules(
    water_box_size, 
    water_molar_mass, 
    CONFIG['water_density_g_cm3']
)

print("\n水分子数の見積もり:")
print(f"  充填領域体積: {water_box_size:.2f} Å³ ({water_box_size * 1e-24:.3e} cm³)")
print(f"  目標密度: {CONFIG['water_density_g_cm3']} g/cm³")
print(f"  水のモル質量: {water_molar_mass:.3f} g/mol")
print(f"  必要な水分子数: {n_molecules_required}")
print(f"  必要な水原子数: {n_molecules_required * 3} (H2O x {n_molecules_required})")
print(f"\n最終構造予測:")
print(f"  既存原子数: {len(pvdf_atoms)}")
print(f"  追加原子数: {n_molecules_required * 3}")
print(f"  総原子数: {len(pvdf_atoms) + n_molecules_required * 3}")

# PVDFは高分子なので密度が高く、水が入る空間が限られる可能性
print(f"\n⚠️ 注意:")
print(f"  PVDFは高分子で密な構造のため、Packmolでも充填に時間がかかる場合があります。")
print(f"  tolerance を大きめ（2.2-2.5 Å）に設定しています。")

## 4. 水分子の充填実行

PVDFは密な構造のため、この処理には時間がかかる場合があります。

In [ ]:
solvated_pvdf = fill_box_with_packmol(
    host_atoms=pvdf_atoms,
    solvent_type='H2O',
    density_g_cm3=CONFIG['water_density_g_cm3'],
    tolerance=CONFIG['tolerance'],
    seed=CONFIG['packmol_seed'],
    structure_cell_size=CONFIG['structure_cell_size'],
    water_fill_cell_size=CONFIG['water_fill_cell_size'],
    verbose=True
)

print("\n" + "=" * 60)
print("  充填完了！")
print("=" * 60)
print(f"最終組成: {solvated_pvdf.get_chemical_formula()}")
print(f"総原子数: {len(solvated_pvdf)}")

# セル情報の表示
cell_params = solvated_pvdf.cell.cellpar()
print(f"\nセル情報:")
print(f"  a={cell_params[0]:.2f} Å, b={cell_params[1]:.2f} Å, c={cell_params[2]:.2f} Å")
print(f"  α={cell_params[3]:.1f}°, β={cell_params[4]:.1f}°, γ={cell_params[5]:.1f}°")
print(f"  体積: {solvated_pvdf.get_volume():.2f} Å³")

## 5. 結果の保存

In [ ]:
# ファイル名の生成
density_str = f"{CONFIG['water_density_g_cm3']:.2f}".replace('.', '_')
output_basename = f"pvdf_h2o_d{density_str}"

# XYZ形式で保存（可視化用）
xyz_path = output_dir / f"{output_basename}_initial.xyz"
write(xyz_path, solvated_pvdf)
print(f"✓ XYZ形式で保存: {xyz_path}")

# CIF形式で保存（結晶構造情報を保持）
cif_path = output_dir / f"{output_basename}_initial.cif"
write(cif_path, solvated_pvdf)
print(f"✓ CIF形式で保存: {cif_path}")

# 統計情報をテキストファイルに保存
stats_path = output_dir / f"{output_basename}_stats.txt"
with open(stats_path, 'w') as f:
    f.write("PVDF + H₂O 溶媒和系の統計情報\n")
    f.write("=" * 60 + "\n\n")
    f.write("設定パラメータ:\n")
    for key, value in CONFIG.items():
        f.write(f"  {key}: {value}\n")
    f.write("\n構造情報:\n")
    f.write(f"  組成: {solvated_pvdf.get_chemical_formula()}\n")
    f.write(f"  総原子数: {len(solvated_pvdf)}\n")
    f.write(f"  セル体積: {solvated_pvdf.get_volume():.2f} Å³\n")
    
    # 実際に追加された水分子数を計算
    n_water_atoms = len(solvated_pvdf) - len(pvdf_atoms)
    n_water_molecules = n_water_atoms // 3
    f.write(f"\n追加された水:\n")
    f.write(f"  水分子数: {n_water_molecules}\n")
    f.write(f"  水原子数: {n_water_atoms}\n")
    
    # 実際の密度
    water_mass_g = (n_water_molecules * 18.015) / units.mol
    vol_cm3 = solvated_pvdf.get_volume() * 1e-24
    actual_density = water_mass_g / vol_cm3
    f.write(f"\n実際の密度:\n")
    f.write(f"  水の密度: {actual_density:.3f} g/cm³\n")
    f.write(f"  目標密度: {CONFIG['water_density_g_cm3']} g/cm³\n")
    f.write(f"  達成率: {actual_density/CONFIG['water_density_g_cm3']*100:.1f}%\n")
    
    # 混合比
    pvdf_atoms_count = len(pvdf_atoms)
    water_atoms_count = n_water_atoms
    f.write(f"\n混合比:\n")
    f.write(f"  PVDF原子数: {pvdf_atoms_count}\n")
    f.write(f"  水原子数: {water_atoms_count}\n")
    f.write(f"  水/(PVDF+水): {water_atoms_count/(pvdf_atoms_count+water_atoms_count)*100:.1f}%\n")

print(f"✓ 統計情報を保存: {stats_path}")

print("\n" + "=" * 60)
print("  全ての処理が完了しました！")
print("=" * 60)
print(f"\n出力ファイル:")
print(f"  1. {xyz_path}")
print(f"  2. {cif_path}")
print(f"  3. {stats_path}")

## 6. （オプション）可視化と分析

In [ ]:
# 原子数の内訳
from collections import Counter
symbols = solvated_pvdf.get_chemical_symbols()
counts = Counter(symbols)

print("最終構造の原子数内訳:")
for symbol, count in sorted(counts.items()):
    print(f"  {symbol}: {count}")

# 水分子の分布を確認（Oの座標）
import matplotlib.pyplot as plt

water_indices = [i for i, s in enumerate(symbols) if s == 'O']
if water_indices:
    water_positions = solvated_pvdf.positions[water_indices]
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    # XY平面
    axes[0].scatter(water_positions[:, 0], water_positions[:, 1], alpha=0.5, s=20)
    axes[0].set_xlabel('X (Å)')
    axes[0].set_ylabel('Y (Å)')
    axes[0].set_title('水分子の分布 (XY平面)')
    axes[0].grid(True, alpha=0.3)
    
    # XZ平面
    axes[1].scatter(water_positions[:, 0], water_positions[:, 2], alpha=0.5, s=20)
    axes[1].set_xlabel('X (Å)')
    axes[1].set_ylabel('Z (Å)')
    axes[1].set_title('水分子の分布 (XZ平面)')
    axes[1].grid(True, alpha=0.3)
    
    # YZ平面
    axes[2].scatter(water_positions[:, 1], water_positions[:, 2], alpha=0.5, s=20)
    axes[2].set_xlabel('Y (Å)')
    axes[2].set_ylabel('Z (Å)')
    axes[2].set_title('水分子の分布 (YZ平面)')
    axes[2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    # 保存
    plot_path = output_dir / f"{output_basename}_distribution.png"
    plt.savefig(plot_path, dpi=150, bbox_inches='tight')
    print(f"\n✓ 分布図を保存: {plot_path}")
    plt.show()
    
    print(f"\n配置された水分子数: {len(water_indices)}")

## 次のステップ

1. **MDシミュレーション**: 作成した構造でMD計算を実行
2. **緩和**: NPT緩和で密度と構造を最適化
3. **反応シミュレーション**: 
   - PVDF-Fと水の相互作用
   - 高温での分解反応
4. **異なる含水率**: `water_density_g_cm3`を変えて比較
5. **界面系**: PVDF/基板/水の3相系の構築

### 推奨設定（Packmol版）

**含水率の調整**:
- **低含水**: `water_density_g_cm3 = 0.3-0.5` (わずかに湿潤)
- **中含水**: `water_density_g_cm3 = 0.6-0.8` (部分的に充填)
- **高含水**: `water_density_g_cm3 = 0.9-1.0` (完全に充填)

**Packmolパラメータ**:
- **tolerance**: `2.2-2.5 Å` (PVDFは密な構造なので大きめに設定)
- **seed**: 固定値で再現性を確保

**セルサイズのカスタマイズ**:
```python
# 例1: PVDF構造は元のセル、水は特定領域のみに充填
CONFIG['structure_cell_size'] = None  # 元のセルサイズを使用
CONFIG['water_fill_cell_size'] = [12.0, 12.0, 12.0]  # 12x12x12 Åの領域に水を充填

# 例2: 大きなセルに拡張して余裕を持たせる
CONFIG['structure_cell_size'] = 35.0  # 35x35x35 Åの立方体セル
CONFIG['water_fill_cell_size'] = 30.0  # 30x30x30 Åの領域に水を充填（周囲5Åは余裕）
```

### Packmolのトラブルシューティング

- **Packmolが見つからない**: `conda install -c conda-forge packmol` でインストール
- **充填に失敗**: 
  - `tolerance`を大きくする（2.5-3.0 Å）
  - `density_g_cm3`を下げる（0.6-0.7 g/cm³）
  - PVDFは密な構造なので、水が入る空間が限られる
- **時間がかかる**: PVDFのような密な構造では正常です。数分待ってください